[Reference](https://medium.com/@sandeep.dhakal/efficiently-dissolving-adjacent-polygons-by-attributes-in-a-large-gis-database-5492e54ef951)

In [1]:
import psycopg
from sqlalchemy import create_engine

# database configuration
user = '<username>'
host = 'localhost'
database = '<dbname>'
driver = 'postgresql+psycopg'
connection_str = f'{driver}://{user}@{host}/{database}'

engine = create_engine(connection_str)

In [2]:
import geopandas as gpd

sql = """
SELECT alumv8, geometry
FROM vic_landuse
WHERE ST_DWithin(geometry,
                    ST_GeomFromText('POINT(144.1 -36.8)', 7844), 0.025)
    AND ST_geometrytype(geometry) = 'ST_Polygon'
"""

sample = gpd.read_postgis(sql, engine, geom_col="geometry")
sample['alumv8'] = sample['alumv8'].astype('category')

ax = sample.plot(
    'alumv8',
    legend=True,
    legend_kwds=dict(loc='upper left',
    bbox_to_anchor=(1.05, 1),
    title='ALUM codes')
)
ax.tick_params('x', rotation=45)

```sql
BEGIN
-- the new table
CREATE TABLE landuse_dissolved (alumv8 int, geometry geometry);

FOR alum_code IN
    SELECT DISTINCT alumv8 FROM sample_landuse
LOOP
    EXECUTE format('DROP VIEW IF EXISTS landuse_%s', alum_code);

    -- create view for the code
    EXECUTE format('
    CREATE VIEW landuse_%s AS
    SELECT * FROM sample_landuse
    WHERE alumv8 = %L',
    alum_code, alum_code);
    
    -- create clustered table
    EXECUTE format(
        'CREATE TABLE landuse_%s_clustered AS
        SELECT
            alumv8,
            ST_ClusterDBSCAN(geometry, 0, 2) OVER() AS cluster_id,
            geometry
        FROM landuse_%s', alum_code, alum_code
    );

    -- dissolve adjacent geometries
    EXECUTE format(
        'CREATE TABLE landuse_%s_dissolved AS
        SELECT alumv8, ST_Union(geometry) as geometry
        FROM landuse_%s_clustered
        GROUP BY alumv8, cluster_id',
        alum_code, alum_code
    );

    -- add dissolved geometries to new table
    EXECUTE format('
        INSERT INTO landuse_dissolved (alumv8, geometry)
        SELECT alumv8, geometry
        FROM landuse_%s_dissolved', alum_code
    );

    -- cleanup intermediary views and tables
    EXECUTE format('DROP TABLE landuse_%s_clustered', alum_code);
    EXECUTE format('DROP VIEW landuse_%s', alum_code);
    EXECUTE format('DROP TABLE landuse_%s_dissolved', alum_code);

    COMMIT;
END LOOP;
END $$
```

In [3]:
original_rows = %sql SELECT COUNT(*) FROM sample_landuse;
new_rows = %sql SELECT COUNT(*) FROM landuse_dissolved;

print(f"original rows = {original_rows[0].count}")
print(f"rows after dissolving = {new_rows[0].count}")

In [4]:
import matplotlib.pyplot as plt

sql = "SELECT * FROM landuse_dissolved"
sample_dis = gpd.read_postgis(sql, engine, geom_col="geometry")
sample_dis['alumv8'] = sample_dis['alumv8'].astype('category')

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)

sample.plot('alumv8', ax=ax1)
sample_dis.plot('alumv8', ax=ax2)

for ax in [ax1, ax2]:
    ax.set_xticks([])
    ax.set_yticks([])